# Optimisation of a function of matrices

$\newcommand{\trace}[1]{\operatorname{tr}\left\{#1\right\}}$
$\newcommand{\Norm}[1]{\lVert#1\rVert}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\inner}[2]{\langle #1, #2 \rangle}$
$\newcommand{\DD}{\mathscr{D}}$
$\newcommand{\grad}[1]{\operatorname{grad}#1}$
$\DeclareMathOperator*{\argmin}{arg\,min}$
Setting up python environment.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.optimize as opt
import scipy.stats as stats
import time

%matplotlib inline

We wish to minimize the following cost function $ f(X) $ defined
over the space of real $ n \times p $ matrices
\begin{equation}
  f(X) = \frac{1}{2} \trace{X^T C X N} + \mu \frac{1}{4} \Norm{N - X^T X}^2_F
\end{equation}
where $ X \in \RR^{n \times p} $, $ n \ge p $, and the matrix $ C $ is symmetric, 
such that $ C = C^T $. The scalar $ \mu $ is assumed to be larger than the $p$th smallest 
eigenvalue of $ C $. The matrix $ N $ is diagonal with distinct positive entries
on the diagonal.
The trace of a square matrix $ A $ is denoted as $ \trace{A} $, and
the Frobenius norm of an arbitrary matrix $ A $ is defined as $ \Norm{A}_F = \sqrt{\trace{A^T A}} $. After some numerical experimentation we eventually compute a closed form of an optimal $X.$



## Frobenious Norm

Implement a Python function ```frobenius_norm``` which accepts an arbitrary matrix $ A $ and returns
$ \Norm{A}_F $ using the formula given. (Use ```numpy.trace``` and ```numpy.sqrt```.)
1. Given a matrix $ A \in \RR^{n \times p} $, what is the complexity of your implementation of ```frobenius_norm```
using the formula above?
2. Can you come up with a faster implementation, if you were additionally told that $ p \ge n $ ?
3. Can you find an even faster implementation than in 1. and 2.? 

### Solution description

In [2]:
def frobenius_norm(A):
    return np.sqrt(np.trace(np.dot(np.transpose(A), A)))

# 1. Complexity is O(p**2 * n)

# 2. Trace of a product is invariant under cyclic permutations. 

def frobenius_norm_fast(A):
    return np.sqrt(np.trace(np.dot(A, np.transpose(A))))

# 3. Sum the squares of the elements of A, which is O(n*p). That's what the native np norm does.

def frobenius_norm_best(A):
    return np.linalg.norm(A)

## Cost Function $f(X)$ with matrix argument

Implement the cost function defined as $f(X)$ above as a function ```cost_function_for_matrix```
in Python.

Hint: As good programmers, we do not use global variables in subroutines.


In [3]:
def cost_function_for_matrix(X, C, N, u):
    trace_term = 0.5 * np.trace( np.transpose(X) @ C @ X @ N )
    norm_term = 0.25 * u * np.linalg.norm( N - np.transpose(X)@ X )**2
    
    return trace_term + norm_term

## Cost Function $f(X)$ with vector argument

Many standard optimisation routines work only with vectors. Fortunately, as vector spaces,
the space of matrices $ \RR^{n \times p} $ 
and the space of vectors $ \RR^{n p} $ are isomorphic. What does this mean?

Implement the cost function $ f(X) $ given $ X $ as a vector and call it ```cost_function_for_vector```.
Which extra arguments do you need for the cost function?


In [4]:
def cost_function_for_vector(X, C, N, u, n, p):
    X_mat = np.array(X).reshape(n,p)
    
    return cost_function_for_matrix(X_mat, C, N, u)

## Construction of a random matrix $C$ with given eigenvalues

A diagonal matrix has the nice property that the eigenvalues can be directly read off
the diagonal. Given a diagonal matrix $ C \in \RR^{n \times n} $ with distinct eigenvalues, 
how many different diagonal matrices have the same set of eigenvalues?

Given a diagonal matrix $ C \in \RR^{n \times n} $ with distinct eigenvalues,
how many different matrices have the same set of eigenvalues?

Given a set of $ n $ distinct real eigenvalues $ \mathcal{E} = \{e_1, \dots, e_n \} $, 
write a Python function ```random_matrix_from_eigenvalues``` which takes a list of
eigenvalues $ E $ and returns a random symmetric matrix $ C $ having the same eigenvalues.

### Solution description

In [64]:
def random_matrix_from_eigenvalues(E):
    O = stats.ortho_group.rvs(dim=len(E)) # Random element of O(n)
    return O @ np.diag(E) @ np.transpose(O)

## Minimising the cost function $f(X)$

Use the minimisation functions ```fmin``` or ```fmin_powell``` provided in the
Python package ```scipy.optimize``` to minimise the cost function ```cost_function_for_vector```.

Hint: Use the argument ```args``` in the minimisation functions  ```fmin``` or ```fmin_powell``` 
to provide the extra parameters to
your cost function ```cost_function_for_vector```.


In [65]:
def argmin_of_cost(cost_function_for_vector, C, N, u, n, p):

    argmin_vector = opt.fmin_powell(cost_function_for_vector, np.zeros(n*p), args = (C, N, u, n, p), xtol = 10**-6)
    return argmin_vector.reshape(n,p)

## Gradient of $f(X)$

Calculate the gradient for the cost function $f(X)$ given the
inner product on the space of real matrices $ n \times p $ is defined as
\begin{equation}
  \inner{A}{B} = \trace{A^T B}
\end{equation}

Implement a function ```gradient_for_vector``` which takes $ X $ as a vector, and
returns the gradient of $ f(X) $ as a vector.


### Solution description

In [66]:
def gradient_for_vector(X, C, N, u, n, p):
    X_mat = np.array(X).reshape(n,p)
    
    trace_term = C @ X_mat @ N
    norm_term_1 = u * X_mat @ np.transpose(X_mat) @ X_mat
    norm_term_2 = u * X_mat @ N
    
    gradient_mat = trace_term + norm_term_1 - norm_term_2
    
    return np.reshape(gradient_mat, n*p)

## Minimising the cost function $f(X)$ using the gradient

Use the minimisation functions ```fmin_cg``` or ```fmin_bfgs``` provided in the
Python package ```scipy.optimize``` to minimise the cost function ```cost_function_for_vector``` utilising the gradient ```gradient_for_vector```.

Compare the speed of convergence to the minimisation with ```fmin``` or ```fmin_powell```.


In [67]:
def argmin_of_cost_2(cost_function_for_vector, C, N, u, n, p):
    
    arg_min_vector= opt.fmin_bfgs(cost_function_for_vector, np.ones(n*p), gradient_for_vector, args = (C, N, u, n, p), gtol=10**-4)
    return arg_min_vector.reshape(n,p)

## Minima of $f(X)$

Compare the columns $x_1,\dots, x_p$ of the matrix $X^\star$ which minimises $ f(X) $ 
\begin{equation}
  X^\star = \argmin_{X \in \RR^{n \times p}} f(X)
\end{equation}

with the eigenvectors related to the smallest eigenvalues of $ C $.


### Solution description

From our numerical experiments below, we observe that for $i=1, \cdots, p$, the column $x_{p-i+1}$ is an eigenvector corresponding to $\lambda_i$, where $\lambda_1, \lambda_2, \cdots, \lambda_n$ is the sequence of eigenvalues of $C$ in ascending order.

$C$ is a real symmetric matrix so it can be diagonalized: $C = P D P^{-1}$ where $P$ is a matrix whose $i$-th column is an eigenvector corresponding to the $i$-th eigenvalue $\lambda_i$.
Further, we can assume that $P$ is an orthogonal matrix : $P^T P = P P^T = I$, i.e. the columns of P are an an orthonormal basis of $\mathbb{R}^n$.

We therefore have the conjecture that $$X^* = [b_1 P_p \ | \ b_2 P_{p-1} \ | \ \ldots \ | \ b_p P_1]
= [P_p \ | \ P_{p-1} \ | \ \ldots \ | \ P_1] \ \operatorname{diag}(b_1, b_2, \ldots , b_p)$$ for some $b_1, \cdots, b_p \in \mathbb{R}$. To find $b_i$, set $$\frac{df}{dX} = CXN + \mu ( X X^T X - XN)$$ to be equal to $0$. 
Some calculation shows $$ \left.\frac{df}{dX}\right|_{X= X^*} =
[P_p \ | \ P_{p-1} \ | \ \ldots \ | \ P_1] * \operatorname{diag}(\ \mu b_1^3 + b_1(\lambda_p n_1 - \mu n_1) \ , \ldots , \  \mu b_p^3 + b_p(\lambda_1 n_p - \mu n_p ) \ )$$

Assume further that $\mu > \operatorname{max}(0, \lambda_p)$. Then $b_i = \sqrt{ n_i \left( 1 - \dfrac{\lambda_{p-i-1}}{\mu} \right)}$ provides a solution.

In [69]:
def argmin_of_f(C, N, u, n, p):
    '''
    Closed form solution for the argmin using the above.
    '''
    vals, vecs = np.linalg.eig(C)
    idx = vals.argsort()[:]
    vals = vals[idx]
    vecs = vecs[:,idx]

    b = [ (N[i,i]/u *(u - vals[p - 1 - i]))**.5 for i in range(p) ]
    X_argmin = np.empty([n,p])

    for i in range(p):
        X_argmin[:,i] = b[i] * vecs[:,p - 1 - i]
    return X_argmin

# Numerical Experiments

Here we use our numerical solver and observe the relation between the columns on $X^*$ (evaluated numerically) and the eigenvalues/eigenvectors of $C,$ which lead to our conjecture for $X^*$ above.

In [441]:
n_ = 6
p_ = 4
C_ = random_matrix_from_eigenvalues([-3, 3 , 7.4 , 5.7, -2.4, -0.3])
u_ = 9
N_ = np.diag([1,3,4,2])

In [442]:
r = argmin_of_f(C_, N_, u_, n_, p_)
#print(r)
cost_function_for_matrix(r, C_, N_, u_)

-28.8825

In [443]:
s = argmin_of_cost(cost_function_for_vector, C_, N_, u_, n_, p_)
#print(s)
cost_function_for_matrix(s, C_, N_, u_)

Optimization terminated successfully.
         Current function value: -39.592531
         Iterations: 34
         Function evaluations: 10335


-39.592531115826219

In [444]:
t = argmin_of_cost_2(cost_function_for_vector, C_, N_, u_, n_, p_)
#print(t)
cost_function_for_matrix(t, C_, N_, u_)

Optimization terminated successfully.
         Current function value: -39.600000
         Iterations: 85
         Function evaluations: 98
         Gradient evaluations: 98


-39.599999999798314

In [445]:
# Eigenvalues (increasing) of C and corresponding eigenvector (in particular, the unit length one)
vals, vecs = np.linalg.eig(C_)
idx = vals.argsort()[:]
vals = vals[idx]
vecs = vecs[:,idx]
print(vals)
print(vecs)

[-3.  -2.4 -0.3  3.   5.7  7.4]
[[ 0.39177197  0.35075259 -0.78666426  0.05855447  0.12398878  0.2929929 ]
 [-0.05317198 -0.4041596  -0.29244146 -0.13476249  0.69482558 -0.49735524]
 [-0.41329929 -0.07999078  0.05139486  0.64516578  0.41385018  0.48232032]
 [-0.32267527 -0.58491359 -0.49563168  0.10160758 -0.54557239  0.0115178 ]
 [-0.36177385 -0.04921734 -0.03185795 -0.74286385  0.16860421  0.53423621]
 [-0.6617186   0.60223156 -0.21524454 -0.00087425 -0.06704932 -0.38551141]]


In [446]:
X_a = argmin_of_cost_2(cost_function_for_vector, C_, N_, u_, n_, p_)
print(X_a)

Optimization terminated successfully.
         Current function value: -39.600000
         Iterations: 85
         Function evaluations: 98
         Gradient evaluations: 98
[[  4.78087538e-02   6.83745606e-01  -9.04756754e-01   1.13090103e+00]
 [ -1.10030710e-01  -7.87852137e-01   1.22793548e-01   4.20412119e-01]
 [  5.26777847e-01  -1.55932990e-01   9.54474247e-01  -7.38846169e-02]
 [  8.29600865e-02  -1.14020735e+00   7.45182861e-01   7.12517092e-01]
 [ -6.06546036e-01  -9.59454326e-02   8.35480747e-01   4.57987625e-02]
 [ -7.14495716e-04   1.17396321e+00   1.52817812e+00   3.09433490e-01]]


In [447]:
for i in range(p_):
    print(X_a[:,i]/X_a[:,i][-1])
    print("")

[ -66.91258282  153.99771862 -737.27222612 -116.10998447  848.91486703
    1.        ]

[ 0.58242507 -0.67110462 -0.13282613 -0.97124624 -0.0817278   1.        ]

[-0.59204928  0.0803529   0.62458311  0.48762827  0.54671686  1.        ]

[ 3.6547467   1.35865099 -0.23877382  2.30265021  0.14800842  1.        ]



In [448]:
for i in range(n_):
    print(vecs[:,i]/vecs[:,i][-1])
    print("")

[-0.59205223  0.08035437  0.62458465  0.48763216  0.54671858  1.        ]

[ 0.58242147 -0.67110332 -0.13282395 -0.97124367 -0.08172493  1.        ]

[ 3.65474669  1.35864754 -0.23877429  2.30264467  0.14800817  1.        ]

[ -66.97715079  154.14721173 -737.9687541  -116.22318068  849.7200684     1.        ]

[ -1.84921756 -10.36290236  -6.1723246    8.13688153  -2.51462963   1.        ]

[-0.76001098  1.29011809 -1.25111815 -0.02987668 -1.38578572  1.        ]



123: 321,
132: 312,
213: 231,
231: 213,
312: 132,
321: 123,
What is this mapping? 4-complement

12: 21,
21: 12,
3-complement

1234 : 4321,
1243 : 4312,
1342 : 4213,
5-complement
